In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Multiple_sclerosis"
cohort = "GSE131281"

# Input paths
in_trait_dir = "../../input/GEO/Multiple_sclerosis"
in_cohort_dir = "../../input/GEO/Multiple_sclerosis/GSE131281"

# Output paths
out_data_file = "../../output/preprocess/Multiple_sclerosis/GSE131281.csv"
out_gene_data_file = "../../output/preprocess/Multiple_sclerosis/gene_data/GSE131281.csv"
out_clinical_data_file = "../../output/preprocess/Multiple_sclerosis/clinical_data/GSE131281.csv"
json_path = "../../output/preprocess/Multiple_sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
from typing import Dict, Any, Optional, Callable

# 1. Gene Expression Data Availability
# Based on the series summary, this dataset contains gene expression data for MS cortical grey matter.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Multiple Sclerosis status):
# From the background information, samples are from MS cases and controls.
# The "ms type" in row 5 can help us identify MS cases vs controls.
# Patient IDs starting with 'M' are MS cases, and those starting with 'C' are controls.
trait_row = 0  # patient id (derived from the first character)

# For age:
# "age at death" is available in row 2
age_row = 2

# For gender:
# "Sex" is available in row 1
gender_row = 1

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> int:
    """Convert patient ID to binary trait (MS = 1, Control = 0)."""
    if value is None:
        return None
    
    # Extract value after "patient id: "
    if "patient id:" in value:
        patient_id = value.split("patient id:")[1].strip()
        # Check if the ID starts with 'M' (MS case) or 'C' (control)
        if patient_id.startswith('M'):
            return 1  # MS case
        elif patient_id.startswith('C'):
            return 0  # Control
    
    return None

def convert_age(value: str) -> Optional[float]:
    """Convert age at death to a continuous value."""
    if value is None:
        return None
    
    if "age at death:" in value:
        try:
            age_str = value.split("age at death:")[1].strip()
            return float(age_str)
        except:
            return None
    
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary (Female = 0, Male = 1)."""
    if value is None:
        return None
    
    if "Sex:" in value:
        sex = value.split("Sex:")[1].strip()
        if sex == 'F':
            return 0  # Female
        elif sex == 'M':
            return 1  # Male
    
    return None

# 3. Save Metadata
# Trait data is available if trait_row is not None
is_trait_available = trait_row is not None

# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # We need to recreate the clinical data from the sample characteristics dictionary
    sample_chars = {
        0: ['patient id: M06', 'patient id: M34', 'patient id: M01', 'patient id: C18', 'patient id: M44', 'patient id: M16', 'patient id: C25', 'patient id: C27', 'patient id: M33', 'patient id: M60', 'patient id: C14', 'patient id: M23', 'patient id: C15', 'patient id: C09', 'patient id: C20', 'patient id: C21', 'patient id: M14', 'patient id: M15', 'patient id: M30', 'patient id: M57', 'patient id: M32', 'patient id: M53', 'patient id: C26', 'patient id: M09', 'patient id: M56', 'patient id: M61', 'patient id: M03', 'patient id: C17', 'patient id: C13', 'patient id: C45'],
        1: ['Sex: F', 'Sex: M'],
        2: ['age at death: 58', 'age at death: 92', 'age at death: 56', 'age at death: 35', 'age at death: 80', 'age at death: 84', 'age at death: 75', 'age at death: 38', 'age at death: 59', 'age at death: 77', 'age at death: 78', 'age at death: 64', 'age at death: 95', 'age at death: 60', 'age at death: 51', 'age at death: 39', 'age at death: 47', 'age at death: 87', 'age at death: 88', 'age at death: 61', 'age at death: 73', 'age at death: 49', 'age at death: 89', 'age at death: 70', 'age at death: 45']
    }
    
    # Convert sample characteristics to a proper DataFrame format for geo_select_clinical_features
    clinical_data = pd.DataFrame(sample_chars)
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(clinical_features)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical features to CSV
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll provide improved code to better locate and parse the data files for this project.

```python
import os
import pandas as pd
import json
import numpy as np
from typing import Optional, Callable, Dict, Any, List
import glob
import re

# Check if gene expression data is likely available
gene_files = glob.glob(os.path.join(in_cohort_dir, "*.txt")) + \
             glob.glob(os.path.join(in_cohort_dir, "*.csv")) + \
             glob.glob(os.path.join(in_cohort_dir, "*matrix*")) + \
             glob.glob(os.path.join(in_cohort_dir, "*Series-Matrix*"))

# Look for files that could contain gene expression data
is_gene_available = False
for file in gene_files:
    # Check file size (gene expression files are typically large)
    if os.path.getsize(file) > 100000:  # 100KB minimum size
        is_gene_available = True
        break

# Look for clinical/sample information using broader patterns
clinical_files = glob.glob(os.path.join(in_cohort_dir, "*sample*.csv")) + \
                 glob.glob(os.path.join(in_cohort_dir, "*clinical*.csv")) + \
                 glob.glob(os.path.join(in_cohort_dir, "*character*.csv")) + \
                 glob.glob(os.path.join(in_cohort_dir, "*phenotype*.csv")) + \
                 glob.glob(os.path.join(in_cohort_dir, "*sample*.txt")) + \
                 glob.glob(os.path.join(in_cohort_dir, "*clinical*.txt")) + \
                 glob.glob(os.path.join(in_cohort_dir, "*pheno*.txt"))

# If no clinical files found, look in series matrix files which might contain clinical data
if not clinical_files:
    matrix_files = glob.glob(os.path.join(in_cohort_dir, "*matrix*")) + \
                   glob.glob(os.path.join(in_cohort_dir, "*Series-Matrix*"))
    for file in matrix_files:
        if os.path.exists(file) and os.path.getsize(file) > 0:
            clinical_files = [file]
            break

clinical_data = pd.DataFrame()
trait_row = None
age_row = None
gender_row = None

# Try to load clinical data if available
if clinical_files:
    for file in clinical_files:
        try:
            if file.endswith('.csv'):
                df = pd.read_csv(file)
            else:  # Assume it's a text file
                # For series matrix files, we need to extract sample characteristics
                with open(file, 'r') as f:
                    lines = f.readlines()
                
                sample_info_lines = []
                in_sample_section = False
                for line in lines:
                    if line.startswith('!Sample_'):
                        in_sample_section = True
                        sample_info_lines.append(line.strip())
                    elif in_sample_section and not line.startswith('!'):
                        in_sample_section = False
                
                if sample_info_lines:
                    # Convert to DataFrame
                    sample_data = []
                    for line in sample_info_lines:
                        parts = line.split('=', 1)
                        if len(parts) == 2:
                            key = parts[0].strip('! \t\n\r')
                            values = parts[1].strip().split('\t')
                            sample_data.append([key] + values)
                    
                    if sample_data:
                        df = pd.DataFrame(sample_data)
                    else:
                        continue
                else:
                    # Try reading as a tab-delimited file
                    df = pd.read_csv(file, sep='\t')
            
            if not df.empty:
                clinical_data = df
                print(f"Clinical data loaded from {file}")
                print("Clinical data preview:")
                print(clinical_data.head())
                break
        except Exception as e:
            print(f"Error reading {file}: {e}")
            continue

    # Check unique values in each row to identify trait, age, and gender information
    unique_values = {}
    for i in range(len(clinical_data)):
        try:
            row_values = clinical_data.iloc[i, 1:].dropna().unique()
            if len(row_values) > 0:
                desc = clinical_data.iloc[i, 0]
                unique_values[i] = {
                    'description': str(desc),
                    'values': [str(v) for v in row_values]
                }
        except:
            continue
    
    print("\nUnique values in sample characteristics:")
    for row, data in unique_values.items():
        print(f"Row {row} - {data['description']}: {data['values']}")
    
    # 2.1 Trait row identification for Multiple Sclerosis
    trait_row = None
    for row, data in unique_values.items():
        desc = str(data['description']).lower()
        values = [str(v).lower() for v in data['values']]
        
        # Check for MS-related terms
        if any(term in desc for term in ['disease', 'ms', 'sclerosis', 'diagnosis', 'status', 'condition', 'group', 'type']) or \
           any('ms' in v or 'multiple sclerosis' in v or 'control' in v or 'patient' in v or 'health' in v for v in values):
            
            # Check if there are multiple categories
            categories = set()
            for v in values:
                if any(term in v for term in ['ms', 'multiple sclerosis', 'patient', 'case']):
                    categories.add('ms')
                elif any(term in v for term in ['control', 'healthy', 'normal']):
                    categories.add('control')
            
            if len(categories) >= 2 or (len(categories) == 1 and len(values) < 3):
                # If we have both categories or just one category with very few samples
                # (suggesting it might be a filter-applied dataset)
                trait_row = row
                break
    
    # 2.2 Define conversion function for trait
    def convert_trait(value):
        if pd.isna(value):
            return None
        
        value = str(value).lower()
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Map to binary values: 1 for MS/patient, 0 for control/healthy
        if any(term in value for term in ['ms', 'multiple sclerosis', 'patient', 'case']):
            return 1
        elif any(term in value for term in ['control', 'healthy', 'normal']):
            return 0
        else:
            return None
    
    # 2.3 Age row identification
    age_row = None
    for row, data in unique_values.items():
        desc = str(data['description']).lower()
        values = data['values']
        
        if 'age' in desc and not 'stage' in desc:  # Avoid confusion with "stage"
            # Check if there are numeric values that could represent ages
            has_numbers = any(re.search(r'\d+', v) for v in values)
            if has_numbers:
                age_row = row
                break
    
    # 2.4 Define conversion function for age
    def convert_age(value):
        if pd.isna(value):
            return None
        
        value = str(value)
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Try to extract numeric age
        try:
            numbers = re.findall(r'\d+\.?\d*', value)
            if numbers:
                return float(numbers[0])
            else:
                return None
        except:
            return None
    
    # 2.5 Gender row identification
    gender_row = None
    for row


### Step 4: Gene Data Extraction

In [ ]:
# 1. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 3. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True


### Step 5: Gene Identifier Review

In [ ]:
# Based on the gene identifiers, I can see they are Illumina probe IDs (prefixed with ILMN_)
# These are not human gene symbols but microarray probe identifiers that need to be mapped
# to standard gene symbols for biological interpretation

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identify the columns in gene annotation for probe IDs and gene symbols
probe_id_col = 'ID'  # Column containing ILMN_* identifiers
gene_symbol_col = 'Symbol'  # Column containing gene symbols

# 2. Get a gene mapping dataframe
# Extract the two columns from the gene annotation dataframe
gene_mapping = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This handles the many-to-many relation between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the shape of the gene expression data after mapping
print(f"Gene data dimensions after mapping: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Print the first few gene symbols to verify the mapping
print("\nFirst 20 gene symbols after mapping:")
print(gene_data.index[:20])

# Save the gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Mapped gene expression data saved to {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the index of gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if clinical data was properly loaded
# First, reload the clinical_data to make sure we're using the original data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print the sample IDs to understand the data structure
print("Sample IDs in clinical data:")
print(clinical_data.columns[:5], "...")  # Show first 5 sample IDs

# Print the sample IDs in gene expression data
print("Sample IDs in gene expression data:")
print(normalized_gene_data.columns[:5], "...")  # Show first 5 sample IDs

# Extract clinical features using the actual sample IDs
is_trait_available = trait_row is not None
linked_data = None

if is_trait_available:
    # Extract clinical features with proper sample IDs
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print(f"Clinical data preview: {preview_df(selected_clinical_df, n=3)}")
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    # Make sure both dataframes have compatible indices/columns
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    
    if linked_data.shape[0] == 0:
        print("WARNING: No samples matched between clinical and genetic data!")
        # Create a sample dataset for demonstration
        print("Using gene data with artificial trait values for demonstration")
        is_trait_available = False
        is_biased = True
        linked_data = pd.DataFrame(index=normalized_gene_data.columns)
        linked_data[trait] = 1  # Placeholder
    else:
        # 3. Handle missing values
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # 4. Determine if trait and demographic features are biased
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
else:
    print("Trait data was determined to be unavailable in previous steps.")
    is_biased = True  # Set to True since we can't evaluate without trait data
    linked_data = pd.DataFrame(index=normalized_gene_data.columns)
    linked_data[trait] = 1  # Add a placeholder trait column
    print(f"Using placeholder data due to missing trait information, shape: {linked_data.shape}")

# 5. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from multiple sclerosis patients, but there were issues linking clinical and genetic data."
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associational studies.")